In [1]:
# Artificial Neural Network

# Installing Theano
# pip install --upgrade --no-deps git+git://github.com/Theano/Theano.git

# Installing Tensorflow
# pip install tensorflow

# Installing Keras
# pip install --upgrade keras

# Part 1 - Data Preprocessing

In [2]:
!pip install -r requirements.txt

In [3]:
# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [4]:
# Importing the dataset
dataset = pd.read_csv('Churn_Modelling.csv')
X = dataset.iloc[:, 3:13].values
y = dataset.iloc[:, 13].values

In [5]:
# Encoding categorical data
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
labelencoder_X_1 = LabelEncoder()
X[:, 1] = labelencoder_X_1.fit_transform(X[:, 1])
labelencoder_X_2 = LabelEncoder()
X[:, 2] = labelencoder_X_2.fit_transform(X[:, 2])
onehotencoder = OneHotEncoder(categorical_features = [1])
X = onehotencoder.fit_transform(X).toarray()
X = X[:, 1:]

/home/alunos/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/home/alunos/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:392: DeprecationWarning: The 'categorical_features' keyword is deprecated in version 0.20 and will be removed in 0.22. You can use the ColumnTransformer instead.
  "use the ColumnTransformer instead.", DeprecationWarning)


In [6]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [7]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [8]:
# Part 2 - Now let's make the ANN!

# Importing the Keras libraries and packages
import keras
from keras.models import Sequential
from keras.layers import Dense

Using TensorFlow backend.


In [9]:
# Initialising the ANN
classifier = Sequential()

# Adding the input layer and the first hidden layer
classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 11))

# Adding the second hidden layer
classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'sigmoid'))

# Adding the output layer
classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'tanh'))

# Compiling the ANN
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

Instructions for updating:
Colocations handled automatically by placer.


In [10]:
# Fitting the ANN to the Training set
classifier.fit(X_train, y_train, batch_size = 10, epochs = 100)

Instructions for updating:
Use tf.cast instead.
Epoch 1/100
8000/8000 [==============================] - 2s 205us/step - loss: 3.2881 - acc: 0.7960
Epoch 2/100
8000/8000 [==============================] - 1s 118us/step - loss: 3.2881 - acc: 0.7960
Epoch 3/100
8000/8000 [==============================] - 1s 134us/step - loss: 3.2881 - acc: 0.7960
Epoch 4/100
8000/8000 [==============================] - 1s 129us/step - loss: 3.2881 - acc: 0.7960
Epoch 5/100
8000/8000 [==============================] - 1s 120us/step - loss: 3.2881 - acc: 0.7960
Epoch 6/100
8000/8000 [==============================] - 1s 110us/step - loss: 3.2881 - acc: 0.7960
Epoch 7/100
8000/8000 [==============================] - 1s 112us/step - loss: 3.2881 - acc: 0.7960
Epoch 8/100
8000/8000 [==============================] - 1s 117us/step - loss: 3.2881 - acc: 0.7960
Epoch 9/100
8000/8000 [==============================] - 1s 124us/step - loss: 3.2881 - acc: 0.7960
Epoch 10/100
8000/8000 [============================

8000/8000 [==============================] - 1s 151us/step - loss: 3.2881 - acc: 0.7960
Epoch 80/100
8000/8000 [==============================] - 1s 131us/step - loss: 3.2881 - acc: 0.7960
Epoch 81/100
8000/8000 [==============================] - 1s 133us/step - loss: 3.2881 - acc: 0.7960
Epoch 82/100
8000/8000 [==============================] - 1s 127us/step - loss: 3.2881 - acc: 0.7960
Epoch 83/100
8000/8000 [==============================] - 1s 125us/step - loss: 3.2881 - acc: 0.7960
Epoch 84/100
8000/8000 [==============================] - 1s 115us/step - loss: 3.2881 - acc: 0.7960
Epoch 85/100
8000/8000 [==============================] - 1s 128us/step - loss: 3.2881 - acc: 0.7960
Epoch 86/100
8000/8000 [==============================] - 1s 130us/step - loss: 3.2881 - acc: 0.7960
Epoch 87/100
8000/8000 [==============================] - 1s 107us/step - loss: 3.2881 - acc: 0.7960
Epoch 88/100
8000/8000 [==============================] - 1s 100us/step - loss: 3.2881 - acc: 0.7960
Epo

In [11]:

# Part 3 - Making predictions and evaluating the model

# Predicting the Test set results
y_pred = classifier.predict(X_test)
y_pred = (y_pred > 0.5)

# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)

In [12]:
print(cm)

[[1595    0]
 [ 405    0]]
